# Question 1

The RNN or Recurrent Neural Network will typically have two inputs. One of the inputs represents the $X(t)$ vector. The second input will be $Y_{t-1}(t)$ which is the output at the previous time $t$. The dimensionality of the input can be of any size. The point of RNN's are to allow for high dimensionality time-series models.

# Question 2

Consider  a  CNN  composed of  three  convolutional  layers,  each with  3  ×  3 kernels,  a stride  of  2,  and  "same"  padding.  The  lowest layer  outputs  100  feature  maps,  the  middle  one outputs 200, and the top one outputs 400. The input images are RGB images of 200 × 300 pixels. What is the total number of parameters in the CNN? If we are using 32-bit floats,  at  least  how much RAM will this network require when making a prediction for a single instance? What about when training on a mini-batch of 50 images?
-------------------------

$Number\ of\ parameters\ = (3 * 3 * 3) * 100 + (3 * 3 * 100 ) * 200 +  (3 * 3 * 200) * 400 = 902,700\ parameters $


$Memory\ Consumption\ of\ single\ image\ = 
    (200 * 300 * 3) * 32 + (100 * 150 * 100) * 32 + (50 * 75 * 200) * 32 + (25 * 37 * 400) * 32 = 89,600,000 * 4 = 358,400,000\ B = 358.4\ MB$
    
$Memory\ Consumption\ of\ minibatch\ = 
    ((200 * 300 * 3) * 32) * 50 + ((100 * 150 * 100) * 32) * 50 + ((50 * 75 * 200) * 32) * 50 + ((25 * 37 * 400) * 32) * 50 = 4,480,000,000 * 4 = 17,920,000,000\ B = 17.92\ GB$

# Question 3

In [30]:
import os
import cv2
from sklearn.utils import shuffle
import numpy as np

In [31]:
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    label = 0
    
    for labels in os.listdir(directory): #Main Directory where each class label is present as folder name.
        if labels == 'glacier': #Folder contain Glacier Images get the '2' class label.
            label = 2
        elif labels == 'sea':
            label = 4
        elif labels == 'buildings':
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'street':
            label = 5
        elif labels == 'mountain':
            label = 3
        
        for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(label)
    
    return shuffle(Images,Labels,random_state=817328462) #Shuffle the dataset you just prepared.

def get_classlabel(class_code):
    labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
    
    return labels[class_code]

In [32]:
Images, Labels = get_images('intel-image-classification/seg_train/seg_train/')

In [33]:
Images = np.array(Images) #converting the list of images to numpy array.
Labels = np.array(Labels)

In [37]:
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer

model = Models.Sequential()

model.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(Layers.MaxPool2D(pool_size=2))
model.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(pool_size=2))
model.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(pool_size=2))
model.add(Layers.Flatten())
model.add(Layers.Dense(128,activation='relu'))
model.add(Layers.Dense(64,activation='relu'))
model.add(Layers.Dense(10,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model.add(Layers.Dense(6,activation='softmax'))

model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 72, 72, 128)       73856     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 70, 70, 128)       147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 31, 31, 256)      

In [40]:
history = model.fit(Images, Labels, epochs=50, validation_split=0.1)

Train on 12630 samples, validate on 1404 samples
Epoch 1/50
  416/12630 [..............................] - ETA: 25:05 - loss: 1.7918 - acc: 0.1587

KeyboardInterrupt: 